In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Activation
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

Using TensorFlow backend.


In [2]:
dt_binary=pd.read_csv('Data/train_binary.csv')
dt_predict=pd.read_csv('Data/test.csv')

In [3]:
dt_predict.head(1)

,Id,Title,Abstract
0,751,Second- and third-generation ALK inhibitors fo...,Crizotinib as the first-generation ALK inhibit...


In [7]:
target_binary=np.multiply([dt_binary['Category']=='cancer'],1).T
text_binary=dt_binary['Abstract']
text_predict=dt_predict['Abstract']

In [8]:
tfidf_vectorizer=TfidfVectorizer(max_df=0.8,min_df=0.002)
text_tfidf=tfidf_vectorizer.fit_transform(text_binary.values).todense()

predict_tfidf=tfidf_vectorizer.transform(text_predict.values).todense()
#text_counts = count_vectorizer.fit_transform(text_binary.values).todense()
#title_counts=count_vectorizer.fit_transform(title_binary.values)

In [9]:
c, r = target_binary.shape
target_binary = target_binary.reshape(c,)
print(predict_tfidf.shape)
print(text_tfidf.shape)
print(target_binary.shape)

(381, 5533)
(2155, 5533)
(2155,)


In [21]:
model=Sequential()
model.add(Dense(200,activation='relu',input_dim=text_tfidf.shape[1]))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [22]:
model.fit(text_tfidf, target_binary, epochs=5, batch_size=64)

Epoch 1/5
2155/2155 [==============================] - 0s - loss: 0.4606 - acc: 0.9007     
Epoch 2/5
2155/2155 [==============================] - 0s - loss: 0.1919 - acc: 0.9698     
Epoch 3/5
2155/2155 [==============================] - 0s - loss: 0.1034 - acc: 0.9796     
Epoch 4/5
2155/2155 [==============================] - 0s - loss: 0.0634 - acc: 0.9847     
Epoch 5/5
2155/2155 [==============================] - 0s - loss: 0.0403 - acc: 0.9903     


In [26]:
yhat=model.predict(predict_tfidf)
yhat

array([[  9.87023711e-01],
       [  9.26503599e-01],
       [  9.98926938e-01],
       [  3.51673365e-02],
       [  3.73468385e-03],
       [  8.81457210e-01],
       [  6.48795485e-01],
       [  3.27092439e-01],
       [  6.38923794e-03],
       [  9.99023080e-01],
       [  1.74999870e-02],
       [  9.88157094e-01],
       [  6.53726421e-03],
       [  6.81892736e-04],
       [  2.10478380e-02],
       [  9.60001815e-03],
       [  3.14056454e-03],
       [  2.05754951e-01],
       [  4.49981494e-03],
       [  1.20264082e-03],
       [  9.99398828e-01],
       [  9.89202023e-01],
       [  9.90249932e-01],
       [  2.46185772e-02],
       [  8.66464436e-01],
       [  6.04339642e-03],
       [  9.60928202e-01],
       [  7.63376383e-03],
       [  1.23298224e-02],
       [  4.89461515e-03],
       [  9.97511983e-01],
       [  6.94893837e-01],
       [  3.87507235e-03],
       [  1.11401209e-03],
       [  9.98197019e-01],
       [  1.43862772e-03],
       [  9.86491978e-01],
 

In [24]:
def create_binary_submission(yhat, ids, save_path):
    if not save_path:
        return "Need a save path!"
    preds = np.round(yhat.flatten())
    sub = pd.DataFrame(data={"Id": ids, "Prediction": preds})

    sub["Category"] = "nonCancer"
    sub.loc[sub["Prediction"] == 1, "Category"] = "cancer"
    path = save_path + "binary_submission.csv"
    sub[["Id", "Category"]].to_csv(path, index=False)

In [25]:
create_binary_submission(yhat,dt_predict['Id'],'Data')

In [63]:
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(target_test, yhat)
metrics.roc_auc_score(target_test,yhat)

0.99698795180722888

In [68]:
yhat_int=np.where(yhat>0.5,1,0)

In [70]:
sum(yhat_int==target_test)/155

array([ 0.97419355])

In [79]:
model.save('NN_1000_1.h5')

K fold cross validation

In [78]:
skf = StratifiedKFold(n_splits=20,shuffle=True)

In [85]:
a = [200]
for x in (a):
    cvscores = []
    for train, test in skf.split(text_tfidf, target_binary):
      # create model
        model=Sequential()
        model.add(Dense(x,activation='relu',input_dim=text_tfidf.shape[1]))
        model.add(Dense(1,activation='sigmoid'))
    
        # Compile model
        model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
        # Fit the model
        model.fit(text_tfidf[train,], target_binary[train], epochs=5, batch_size=64)
        # evaluate the model
        scores = model.evaluate(text_tfidf[test], target_binary[test], verbose=0)
        print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
        cvscores.append(scores[1] * 100)
        print("%d %.2f%% (+/- %.2f%%)" % (x, np.mean(cvscores), np.std(cvscores)))

Epoch 1/5
2046/2046 [==============================] - 4s - loss: 0.4787 - acc: 0.8856     
Epoch 2/5
2046/2046 [==============================] - 0s - loss: 0.2077 - acc: 0.9677     
Epoch 3/5
2046/2046 [==============================] - 0s - loss: 0.1115 - acc: 0.9780     
Epoch 4/5
2046/2046 [==============================] - 0s - loss: 0.0684 - acc: 0.9844     
Epoch 5/5
2046/2046 [==============================] - 0s - loss: 0.0442 - acc: 0.9883     
acc: 96.33%
200 96.33% (+/- 0.00%)
Epoch 1/5
2046/2046 [==============================] - 4s - loss: 0.4758 - acc: 0.9042     
Epoch 2/5
2046/2046 [==============================] - 0s - loss: 0.2116 - acc: 0.9668     
Epoch 3/5
2046/2046 [==============================] - 0s - loss: 0.1146 - acc: 0.9780     
Epoch 4/5
2046/2046 [==============================] - 0s - loss: 0.0701 - acc: 0.9829     
Epoch 5/5
2046/2046 [==============================] - 0s - loss: 0.0446 - acc: 0.9897     
acc: 96.33%
200 96.33% (+/- 0.00%)
Epoch 1/5
